In [1]:
from functions import *
import xml.etree.ElementTree as ET
import os
import pandas as pd
from lxml import etree
import sys

In [2]:
regions_dict = {'nom':'BT5137', 'details':'BT5138', 'num_mat':'BT5139', 'signalement':'BT5140',
                'affectation':'BT5141', 'instruction':'BT5142', 'adresse':'BT5143', 'dates':'BT5144', 
                'etat_civil':'BT5145', 'decision':'BT5146'}

In [3]:
for key, value in regions_dict.items():
    print(key + " : " + value)

nom : BT5137
details : BT5138
num_mat : BT5139
signalement : BT5140
affectation : BT5141
instruction : BT5142
adresse : BT5143
dates : BT5144
etat_civil : BT5145
decision : BT5146


In [6]:
directory = "/Users/Theo/Desktop/big_drop_v2/"

In [11]:
test1 = pd.read_csv("/Users/Theo/dev/matricule_htr/output/sample_table.tsv", sep="\t")

In [12]:
test1

,filename,nom,etat_civil,decision,details,campagnes,num_mat,signalement,affectation,adresse,cassier,blessures,instruction
0,pinero_ocr.xml,"['Vern', 'Moger', 'pouers', 'Anguote']","['Cit', 'de', '20e', 'Aout', '1895.', 'Carbro'...","['1', '24', 'Gabrurel']","['Incorporé', 'au', '14e', 'rég^t', 'd', 'Art...","['intérieer', 'du', '16-1!p!', '1226-1', 'arr...","['Re', '28', '2211']","['elixtrivs', 'oovile', '28']","['O^lis', 'le', '19-10-34']","['15', 'segéce', 'de', 'MM^e', ""S'orveeau"", 'M...",NaN,NaN,NaN
1,ED_ocr.xml,"['Marie', 'François', 'Edouard']","['1', 'Novembre', '1898', 'a', 'Bouscal', 'e',...","['ReM', 'FORe', 'RéRET', '1', 'NR', 'RECROTRNE...","['Engage', 'volontaire', 'pour', 'la', 'duxée...",NaN,"['O', '1917', 'AAMR']",NaN,"['CU', 'DI', '194', '58^e', 'RAC', '7248', '22...","['41', '12-21', '25', 'Rue', 'deMuchouse', 'B^...",NaN,NaN,NaN
2,pinero2_ocr.xml,"['NoGC', '1', 'aurice', 'Henri', 'Maccel']","['29', 'novembre', '1900', 'Dhicachon', 'e', '...","['18', 'Arcachan', '31', '1ere']","['Incorposé', 'à', 'compter', 'de', '15', 'M...","['Marsé', 'en', 'Guerre', '11-4-1220', 'u', '...","['', '2', '11', '1720']",['60'],"['régt', ""d'avécation"", 'de', 'Mraroc', '37^e...",NaN,"['', '', ' ', 'Sosuert', 'hoble', ' ', '8', '2...",NaN,NaN
3,yvelines_ocr.xml,"['e', 'I-', '', 'Marie', 'Jean', 'dooéé']","['d', '19', 'Février', '1901', 'Vesoul', 'e',...","['221', 'e', 'Versall', 'de', 'Vord', '196']","['Arrivé', 'le', '22', 'Avril', '1918', 'au',...",NaN,"['2901', '1919']","['chatains', 'Pruns', ' ', '18', '7', '69']","['2e', 'Dépot', 'des', 'Taupissée', 'de', 'la...",['AADITEES'],NaN,[''],"['', 'bentenmement']"


In [14]:
output = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        # Load the XML file into an ElementTree object
        tree = ET.parse(os.path.join(directory, filename))
        
        result = {}
        result['filename'] = filename
        for key, value in regions_dict.items():
            line1 = ".//{http://www.loc.gov/standards/alto/ns-v4#}"
            line2 = f"TextBlock[@TAGREFS='{value}']/"
            lines = tree.findall(line1 + line2 + "{http://www.loc.gov/standards/alto/ns-v4#}TextLine")
        
            cat = []
        
            for content in lines:
                cat.append(content.find('.//{http://www.loc.gov/standards/alto/ns-v4#}String').attrib['CONTENT'])
            
            result[f'{key}'] = cat
            df_dictionary = pd.DataFrame([result])
            
        output = pd.concat([output, df_dictionary], ignore_index=True)

In [15]:
output['instruction_militaire'] = output['instruction'].apply(extract_intruction_militaire)
output['instruction'] = output['instruction'].apply(extract_digit_or_x)
output['taille'] = output['signalement'].apply(extract_height)
output['prenom'], output['nom'] = zip(*output['nom'].apply(extract_first_name))
output['date_naissance'] = output['etat_civil'].apply(extract_date)
output['num_mat'], output['classe'] = zip(*output['num_mat'].apply(lambda x: (x, extract_classe(x))))
output['num_mat'] = output['num_mat'].apply(clean_num_mat)
output['details'] = output['details'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)



In [16]:
order = ['filename', 'nom', 'prenom', 'date_naissance', 'num_mat', 'classe', 'signalement', 'taille', 
         'instruction', 'instruction_militaire', 'details', 'affectation', 'adresse', 
         'dates', 'etat_civil', 'decision']

output = output[order]

In [17]:
output.sample(n=40)

,filename,nom,prenom,date_naissance,num_mat,classe,signalement,taille,instruction,instruction_militaire,details,affectation,adresse,dates,etat_civil,decision
74,125912_archives_FRAD075RM_D4R1_2032_0198_D.xml,Hernlen,Louis Fernand,16novembre1899,5599,NaN,"[bl., long, marr., vert., rect., 78]",178.0,None,None,Hernlen1918/55991eR A12 6.4019185599Blessé le ...,"[115^e Reg^t Infanterie 14757, 156° Rég^t Infa...","[31-3-31. Pantin 7 rue de la Marine, f^le Pant...",[],"[Paris, 16 Novembre 1899, 20^e arr^t, la Seine...",[]
109,245309_archives_FRAD075RM_D4R1_1054_0751_D.xml,Gesmier,"Jean, Baptiste, Auguste, Constant",11février1879,3994,NaN,"[et, Châtains clair, bleus, large, moyen, moy...",166.0,3,None,Dirigé le 1novembre 1900 sur le. 151^e régime...,"[151^e régiment d'Infanterie, CM 22^c S^on c...","[9 rue de Cheroy 17, 26/2/199 19 Ru de Chèroy...",[1^er Nov^bre 1^er Nov^bre 1^er Nov^bre 1^er N...,"[Ducey, 11 février 1879, la Manche, udit, Par...","[Bon pour le service armé, 1ère]"
20,219385_archives_FRAD075RM_D4R1_1052_0443_D.xml,Dany,"Alfred, Gilbert",14janvier1879,2814,1897.0,"[noirs, ovale, noirs, haut, 69, moyen]",169.0,3,None,Engagé Volontaire pour 4 ans le 17 Janvier 189...,"[146^e Régt dinfanterie -, 11^e Batl^on de Ch...","[10 Janvier 1903 Paris 16 Rue de la Chapelle, ...",[],"[Paris, 14 Janvier 1879, e la Seine, u 18^e ar...","[Engagé Volontaire, 490, u 18^e Arrondiss^t de..."
36,125912_archives_FRAD075RM_D4R1_2032_0194_D.xml,Hernlen,Louis Fernand,16novembre1898,5599,NaN,"[bl., long, marr., vert., rect., 78]",178.0,None,None,Incorporé à compter du 16 avril 1917 arrivé au...,"[115^e Reg^t Infanterie 14959, 156° Rég^t Inf...","[31-3-31 Pantin 7 rue de la Marine, f^le Panti...",[],"[16 novembre 1898, Paris, 20^e arr^t, la Seine...","[Aubervilliers, 17]"
90,701648_archives_FRAD075RM_D4R1_0542_0337_D.xml,Chesneau,"Gaston, Adrien",23juin1868,2274,NaN,"[et, chât-gris, gris - bleus, moyen, relevé, m...",168.0,3,None,Partie le 13 novembre 1889 pour le 3e régimen...,"[3^e rég^t du Genie, Régiment de Réserv, du...","[Se retire a Paris 21 rue de Gergovie, 14 8^br...",[1^e 9^bre 1892 1^e 9^bre 1899 1^e 9^bre 1908 ...,"[23 Juin 1868, Paris, e la seine, 14° arrond^t...","[Bon, 1]"
63,536388_archives_FRAD075RM_D4R1_0779_0173_D.xml,Maréchal,Henri Lévis,21décembre1874,1107,1892.0,"[et, bruns, gris, ordinaire, long, moyenne, ro...",166.0,x,None,Engagé Volre pour 3 ans le 5 Novembre 1893 a A...,"[10e Cie d'ouv d'artie, 26e d'artillerie, Rég ...","[17 8bre 97 Aubervilliers, 3 rue Crévet, k, 28...","[5 9bre 5 9bre 5 9bre 5 9bre, 1896 1906 1912 1...","[21 Décembre 1874,, Aubervilliers, u dit, e l...","[Engagé Volontaire pour 3ans, 4]"
67,830203_archives_FRAD075RM_D4R1_1876_0225_D.xml,Mallerin,Georges,19mai1895,16012,1911.0,"[chatain, chatain, ord^e, ord., ovale, 68]",168.0,3,None,Incorporé a compter du 19 Décembre 1914. Arr...,"[103^e Régiment d'Infanterie\ 8232, 170^e ____...",[g^e 3.3.28 d'es Paris 11^e 26 rue de la Folie...,[],"[19 Mai 1895, Paris, 11^e Arrondt, Seine, Pari...","[825, u 11^e Arrondt, 1, 14]"
61,830887_archives_FRAD075RM_D4R1_1971_0419_D.xml,Cottin,Marcel,None,1168,NaN,"[brun, brun, long, rouge, rectiligne, 63]",163.0,None,None,3148^e Régiment d'Infanterie 13163116^e Régime...,[d'Inf^ri],"[armontel, Saus 28^e]",[],[],[]
86,203873_archives_FRAD075RM_D4R1_1668_0711_D.xml,Jeannin,Henri Louis,9janvier1892,4883,NaN,"[blonds, bleus, moyen fuyant, gros, long, 72]",172.0,3,None,Incorporé à compter du 8 Octobre 1913corps le ...,"[3^e Reg^t de Genie 7607, 91^e Reg^t D'infant...","[915 Chatellerault Rue des 3 Pigeons 32, retir...",[],"[Port marly, 9 Janvier 1892, e Seine et Oise, ...","[297, e s^t denis, 1, 13]"
50,830887_archives_FRAD075RM_D4R1_1971_0417_D.xml,Cottin,Marcel,4janvier1897,1168,NaN,"[brun, long, brun, large, 63, rectiligne]",163.0,3,None,Cf Allemagne 1914 . 1918,"[148^e Régiment d'Infanterie 13163, 116^e Rég...","[CC 13.6.34 confirmé a Alger ?? ), CC 11.3.38 ...",[],"[4 Janvier 1897, Aubervilliers, Aubervillier

In [18]:
output.to_excel('/Users/Theo/Desktop/sample_table.xlsx', index=False)

/var/folders/41/1r7_f1_93g56d738vfqw0y8m0000gn/T/ipykernel_83134/2716810977.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  output.to_excel('/Users/Theo/Desktop/sample_table.xlsx', index=False)


In [19]:
output.to_csv('/Users/Theo/Desktop/sample_table.tsv', sep = '\t', index=False)